In [2]:
import os
import sys
from dateutil.parser import parse
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [6]:
consum = pd.read_csv('../data/consumption.txt')
consum.drop(['id'], axis=1,inplace=True)
consum.fillna('',inplace=True)



In [5]:
sys.path

['',
 '/home/liwenjie/anaconda3/lib/python36.zip',
 '/home/liwenjie/anaconda3/lib/python3.6',
 '/home/liwenjie/anaconda3/lib/python3.6/lib-dynload',
 '/home/liwenjie/anaconda3/lib/python3.6/site-packages',
 '/home/liwenjie/anaconda3/lib/python3.6/site-packages/IPython/extensions',
 '/home/liwenjie/.ipython']